# Paris' trees analysis
## Objective:

Define a better inspection route for gardeners to take care of the city's trees

With the identification of types and caracteristics, routes could be planned to minimize mouvements and optimize the work force.
## Caracteristics:

What makes two trees alike / different ? Size ? Height ? Health ? Location ?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('p2-arbres-fr.csv', sep = ";")

In [4]:
data.head(5)

,id,type_emplacement,domanialite,arrondissement,complement_addresse,numero,lieu,id_emplacement,libelle_francais,genre,espece,variete,circonference_cm,hauteur_m,stade_developpement,remarquable,geo_point_2d_a,geo_point_2d_b
0,99874,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,19,Marronnier,Aesculus,hippocastanum,NaN,20,5,NaN,0.0,48.857620,2.320962
1,99875,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,20,If,Taxus,baccata,NaN,65,8,A,NaN,48.857656,2.321031
2,99876,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,21,If,Taxus,baccata,NaN,90,10,A,NaN,48.857705,2.321061
3,99877,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,22,Erable,Acer,negundo,NaN,60,8,A,NaN,48.857722,2.321006
4,99878,Arbre,Jardin,PARIS 17E ARRDT,NaN,NaN,PARC CLICHY-BATIGNOLLES-MARTIN LUTHER KING,000G0037,Arbre à miel,Tetradium,daniellii,NaN,38,0,NaN,NaN,48.890435,2.315289


In [ ]:
data.describe()

In [ ]:
data.info()

## First glance :
### Structure:

    18 columns
    200137 lines

Now to print a detail of each column

In [9]:
max_value=200137

for col in data:
    unique_counter = data[col].nunique()
    if unique_counter == max_value:
        print("Only unique values")
    else:
        count = data[col].value_counts()
        freq = data[col].value_counts(normalize=True)

        dat = {"Count" :count , "Freq":freq}

        df = pd.DataFrame(dat)
        print({col},"Unique_counter: ", unique_counter, "\n" ,df)

Only unique values
{'type_emplacement'} Unique_counter:  1 
                    Count  Freq
type_emplacement              
Arbre             200137   1.0
{'domanialite'} Unique_counter:  9 
                Count      Freq
domanialite                   
Alignement    104949  0.524388
Jardin         46262  0.231153
CIMETIERE      31926  0.159522
DASCO           6422  0.032088
PERIPHERIQUE    5327  0.026617
DJS             3900  0.019487
DFPE            1325  0.006620
DAC               21  0.000105
DASES              4  0.000020
{'arrondissement'} Unique_counter:  25 
                    Count      Freq
arrondissement                    
PARIS 15E ARRDT    17151  0.085696
PARIS 13E ARRDT    16712  0.083503
PARIS 16E ARRDT    16403  0.081959
PARIS 20E ARRDT    15340  0.076647
PARIS 19E ARRDT    13709  0.068498
PARIS 12E ARRDT    12600  0.062957
SEINE-SAINT-DENIS  11570  0.057810
BOIS DE VINCENNES  11510  0.057511
PARIS 14E ARRDT    11399  0.056956
PARIS 17E ARRDT    10762  0.053773
PARIS 1